In [2]:
from __future__ import annotations

import json
import glob #Search files
import re
import os #Path things
import pickle
from typing import Tuple

import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader, random_split

from dataloader_wrapper import DataloaderWrapper

In [21]:
import torch
import numpy as np

class ConvBlock(torch.nn.Module):
    def __init__(self, input_img_size, input_channels, output_channels, dropout_rate) -> None:
        super().__init__()

        self._conv = torch.nn.Conv2d(input_channels, output_channels, kernel_size=(3,3), stride=1, padding=1)
        self._dropout = torch.nn.Dropout(dropout_rate)
        self._relu = torch.nn.ReLU()
        #self._max_pool = torch.nn.MaxPool2d(kernel_size=(2,2))
        self._batch_norm = torch.nn.BatchNorm2d(output_channels)
        
        self._conv1 = torch.nn.Conv2d(input_channels, output_channels, kernel_size=1)

        self._output_channels = output_channels

        self._output_img_size = input_img_size
        
        #Conv
        self._output_img_size[0] = ((self._output_img_size[0] + (2*1) - 1* (3-1)-1)/1)+1
        self._output_img_size[1] = ((self._output_img_size[1] + (2*1) - 1* (3-1)-1)/1)+1

        #Max pool
        #self._output_img_size[0] = ((self._output_img_size[0] + (2*0) - 1* (2-1)-1)/2)+1
        #self._output_img_size[1] = ((self._output_img_size[1] + (2*0) - 1* (2-1)-1)/2)+1

    def forward(self, x):
        y1 = self._conv(x)
        y1 = self._dropout(y1)
        y1 = self._relu(y1)
        #y1 = self._max_pool(y1)

        y2 = self._conv1(x)

        y = y1+y2

        y = self._batch_norm(y)

        return y

In [22]:
conv_block = ConvBlock([32, 32], 3, 20, 0.1)

In [23]:
x = torch.empty((1, 32, 32, 3))
x_torch = x.permute(0, 3, 1, 2)

In [24]:
y = conv_block(x_torch)

In [25]:
x_torch.shape, y.shape

(torch.Size([1, 3, 32, 32]), torch.Size([1, 20, 32, 32]))

In [28]:
conv_block._output_channels, conv_block._output_img_size

(20, [32.0, 32.0])

## Dataset

In [2]:
dataset_path = os.environ["USERPROFILE"]+"\\AppData\\LocalLow\\DefaultCompany\\IA904-3D_Pose\\solo"
dataset_path = "I:\\.shortcut-targets-by-id\\1S6q0nt4z5LYa-5VkpC8qxag2b_jjO6e9\\IA904\\Dataset"

In [3]:
features_dataset_path = os.path.join(dataset_path, "extracted_features")
index_path = os.path.join(dataset_path, "index.pickle")

In [4]:
with open(index_path, "rb") as index_file:
    dataset_index = pickle.load(index_file)

In [5]:
dataset_index = pd.DataFrame(dataset_index)

In [6]:
coco2unity =   {"nose": 87, #nose
               "left_eye": 80, #eye_left
               "right_eye": 81, #eye_right
               "left_ear": 85, #ear_left
               "right_ear": 86, #ear_right
               "left_shoulder": 21, #shoulder_left
               "right_shoulder": 50, #shoulder_right
               "left_elbow": 22, #elbow_left
               "right_elbow": 51, #elbow_right
               "left_wrist": 25, #wrist_left
               "right_wrist": 54, #wrist_right
               "left_hip": 88, #hip_left
               "right_hip": 89, #hip_right
               "left_knee": 2, #knee_left
               "right_knee": 10, #knee_right
               "left_ankle": 4, #ankle_left
               "right_ankle": 12, #ankle_right
            }

In [7]:
interest_indexes = list(coco2unity.values())

In [8]:
nan_count = 0
for i in range(len(dataset_index)):
    nan_count += np.isnan(dataset_index["keypoints_3d"][0][interest_indexes]).sum()

nan_count

0

In [9]:
class SyntheticOAKDDataset(Dataset):
    def __init__(self, dataset_index:pd.DataFrame, 
                 keypoint_mapping : dict[str, int], 
                 scenarios:list[int]) -> None:
        super().__init__()

        self._dataset_index = dataset_index
        self._keypoint_mapping = keypoint_mapping
        self._keypoint_indexes = list(keypoint_mapping.values())
        self._scenarios = scenarios

        self._n_keypoint_per_image = len(keypoint_mapping)

        n_sample = 0
        n_sample = []
        for scenario in scenarios:
            n_sample.append((dataset_index["scenario"] == scenario).sum())

        self._n_sample = np.array(n_sample)
        self._n_sample_cumsum = np.cumsum(n_sample)

    def __len__(self) -> int:
        return self._n_sample.sum()
    
    def __getitem__(self, index) -> Tuple[torch.Tensor, torch.Tensor]:
        img_index = index // self._n_keypoint_per_image
        keypoint_index = index % self._n_keypoint_per_image

        scenario_index = np.argwhere(self._n_sample_cumsum > img_index)[0][0]
        scenario = self._scenarios[scenario_index]

        index_in_scenario = img_index
        if scenario_index != 0:
            index_in_scenario -= self._n_sample_cumsum[scenario_index-1]

        scenario_mask = self._dataset_index["scenario"] == scenario
        df = self._dataset_index[scenario_mask]
        line = df.iloc[[index_in_scenario]]
        
        # Get features
        sequence = line["sequence"].values[0]
        step = line["step"].values[0]

        features_folder = os.path.join(features_dataset_path,
                               f"Scenario{scenario}",
                               f"sequence.{sequence}")
        
        features = []
        for name in ["left", "right", "center"]:
            features_path = os.path.join(features_folder, 
                                        f"step{step}.camera_{name}_heatmaps.npz")
            
            camera_features = np.load(features_path)["img_heatmaps"]
            keypoint_features = camera_features[keypoint_index]
            features.append(keypoint_features)
        
        features = np.array(features)

        # Get target

        keypoints_3d = df.iloc[[1]]["keypoints_3d"].values[0]
        unity_keypoint_index = self._keypoint_indexes[keypoint_index]
        keypoints_3d = keypoints_3d[unity_keypoint_index]   

        return torch.tensor(features, dtype=torch.float32), torch.tensor(keypoints_3d, dtype=torch.float32)

In [10]:
scenario0_dataset = SyntheticOAKDDataset(dataset_index, coco2unity, [0])

In [11]:
train_dataset, validation_dataset = random_split(scenario0_dataset, [0.8, 0.2])

In [12]:
import multiprocessing as mp

mp.cpu_count()

12

In [13]:
loader = DataloaderWrapper(scenario0_dataset, 1, mp.cpu_count()-1, True).dataloader

In [14]:
import time

start = time.time()

for inputs, targets in tqdm.tqdm(loader):
    pass

end = time.time()

  0%|          | 0/2000 [00:00<?, ?it/s]

## Model